- [Online IRT MCMC + Moment Matching](#MCMC-+-Moment-Matching)
- [Online IRT Search](#Search)

In [1]:
import probpy as pp
import numpy as np
import numba
import random

Utilities and data
---

In [2]:

@numba.jit(nopython=True, fastmath=True, forceobj=False)
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def logit(x):
    return np.log(x / (1 - x))

student_skill = logit(0.7)

items = logit(np.array([0.4, 0.6, 0.8, 0.7]))  # difficulties


fast_p = pp.normal.fast_p
def likelihood(obs, item, skill): # IRT likelihood
    return fast_p(obs - sigmoid(skill - item), mu=0.0, sigma=0.6)
   
    
## IRT samples
samples = 100
obs, its = [], []
for i in range(samples):  
    item = items[np.random.randint(0, items.size)]
    outcome = float(np.random.rand() < sigmoid(student_skill - item))

    obs.append(outcome)
    its.append(item)

MCMC + Moment Matching
---

In [4]:
%%time


prior_skill = pp.normal.med(mu=0.0, sigma=5)

for i in range(samples)[:30]:
    prior_skill = pp.parameter_posterior((obs[i], its[i]), likelihood=likelihood, prior=prior_skill,
                                         mode="mcmc", match_moments_for=pp.normal,
                                         samples=20000, mixing=10000, batch=5, energy=0.1)
        
    
    print("observation", obs[i], "item", sigmoid(its[i]), "mode", sigmoid(pp.mode(prior_skill)))


observation 1.0 item 0.6 mode 0.6225059514783556
observation 1.0 item 0.6 mode 0.6554780171083098
observation 1.0 item 0.4 mode 0.45999009663243956
observation 0.0 item 0.7 mode 0.3630372401468977
observation 0.0 item 0.6 mode 0.32824264854035146
observation 1.0 item 0.4 mode 0.4279925276803948
observation 1.0 item 0.4 mode 0.4542575933060088
observation 0.0 item 0.4 mode 0.4024117973178125
observation 1.0 item 0.6 mode 0.5169701060098407
observation 1.0 item 0.4 mode 0.6162432913004154
observation 1.0 item 0.6 mode 0.6300152747562603
observation 0.0 item 0.7 mode 0.6467488944583252
observation 1.0 item 0.7 mode 0.7099893186899374
observation 1.0 item 0.6 mode 0.722954678778478
observation 0.0 item 0.8 mode 0.7119169284323583
observation 0.0 item 0.8 mode 0.7129548634722916
observation 0.0 item 0.7 mode 0.7061706084108672
observation 0.0 item 0.6 mode 0.7237591685722826
observation 1.0 item 0.4 mode 0.7404711013701378
observation 0.0 item 0.7 mode 0.7222712268824201
observation 0.0 ite

Search
---

In [26]:
%%time


prior_skill = pp.normal.med(mu=0.0, sigma=10)

for i in range(samples)[:30]:
    prior_skill = pp.parameter_posterior((obs[i], its[i]), 
                                         likelihood=likelihood, priors=prior_skill,
                                         mode="search",
                                         samples=300, batch=5,
                                         volume=100, energy=0.1,
                                         variance=2.0)
    modes = sigmoid(np.array(pp.mode(prior_skill)))
    
    print("observation", obs[i], "item", sigmoid(its[i]), "mode", sigmoid(pp.mode(prior_skill))[0])



observation 1.0 item 0.6 mode [0.76307783]
observation 1.0 item 0.4 mode [0.79026422]
observation 1.0 item 0.7 mode [0.8579251]
observation 0.0 item 0.8 mode [0.73040998]
observation 0.0 item 0.8 mode [0.61799228]
observation 0.0 item 0.8 mode [0.53587315]
observation 0.0 item 0.8 mode [0.47865744]
observation 1.0 item 0.7 mode [0.71926162]
observation 0.0 item 0.8 mode [0.54868546]
observation 1.0 item 0.6 mode [0.73544899]
observation 1.0 item 0.4 mode [0.77172806]
observation 1.0 item 0.6 mode [0.83478771]
observation 0.0 item 0.8 mode [0.70176608]
observation 1.0 item 0.7 mode [0.82534356]
observation 0.0 item 0.8 mode [0.69042595]
observation 1.0 item 0.7 mode [0.81443007]
observation 1.0 item 0.4 mode [0.82775929]
observation 1.0 item 0.7 mode [0.87594142]
observation 0.0 item 0.8 mode [0.750519]
observation 1.0 item 0.6 mode [0.82513622]
observation 1.0 item 0.8 mode [0.88647549]
observation 1.0 item 0.4 mode [0.89555844]
observation 0.0 item 0.7 mode [0.74407424]
observation 0.

Different model
---

In [29]:

def likelihood(obs, diff, know):
    result = []
    for k in know:
        result.append(pp.normal.p(obs - np.maximum(k - diff, 0.0), mu=0.0, sigma=0.1))
    
    return np.array(result)


prior_skill = pp.normal.med(mu=0.0, sigma=10)

for i in range(samples)[:30]:
    prior_skill = pp.parameter_posterior((obs[i], its[i]), 
                                         likelihood=likelihood, priors=prior_skill,
                                         mode="search",
                                         samples=300, batch=5,
                                         volume=100, energy=0.1,
                                         variance=1.0)
    modes = sigmoid(np.array(pp.mode(prior_skill)))
    
    print("observation", obs[i], "item", sigmoid(its[i]), "mode", sigmoid(pp.mode(prior_skill))[0])

observation 1.0 item 0.6 mode [0.81213738]
observation 1.0 item 0.4 mode [0.67756136]
observation 1.0 item 0.7 mode [0.84057223]
observation 0.0 item 0.8 mode [0.80633885]
observation 0.0 item 0.8 mode [0.8008372]
observation 0.0 item 0.8 mode [0.80013086]
observation 0.0 item 0.8 mode [0.8003221]
observation 1.0 item 0.7 mode [0.85298123]
observation 0.0 item 0.8 mode [0.81167636]
observation 1.0 item 0.6 mode [0.80194423]
observation 1.0 item 0.4 mode [0.67789726]
observation 1.0 item 0.6 mode [0.78329293]
observation 0.0 item 0.8 mode [0.78232612]
observation 1.0 item 0.7 mode [0.85138806]
observation 0.0 item 0.8 mode [0.81066259]
observation 1.0 item 0.7 mode [0.85489143]
observation 1.0 item 0.4 mode [0.69012618]
observation 1.0 item 0.7 mode [0.8412959]
observation 0.0 item 0.8 mode [0.80642762]
observation 1.0 item 0.6 mode [0.80395207]
observation 1.0 item 0.8 mode [0.90264561]
observation 1.0 item 0.4 mode [0.70527953]
observation 0.0 item 0.7 mode [0.70281714]
observation 0.